In [3]:
import pandas as pd 
import numpy as np
from dash_website import (
    DIMENSIONS,
    ALL_BIOMARKERS,
    ALL_ENVIRONMENTAL,
    ALL_SOCIOECONOMICS,
    ALL_PHENOTYPES,
    ALL_DISEASES,
    ALL_CATEGORIES,
    CATEGORIES,
)

missing_correlations = []

pd.read_csv(f"../../data/page6_LinearXWASCorrelations/correlations/Correlations_Intersection_Spearman.csv", index_col=[0])

,env_dataset,organ_1,organ_2,corr,sample_size
0,Alcohol,*instances01,*instances01,1.000000,2
1,Alcohol,*instances01,*instances1.5x,1.000000,2
2,Alcohol,*instances01,*instances23,NaN,1
3,Alcohol,*instances01,Abdomen,NaN,1
4,Alcohol,*instances01,AbdomenLiver,1.000000,2
...,...,...,...,...,...
96916,All_Diseases,\*,MusculoskeletalKnees,-0.485714,6
96917,All_Diseases,\*,MusculoskeletalFullBody,0.190769,25
96918,All_Diseases,\*,MusculoskeletalScalars,0.686697,224
96919,All_Diseases,\*,PhysicalActivity,0.984152,235


In [17]:
string = ""
for d in ALL_CATEGORIES:
    string += f'"{d}" '
string

'"Alcohol" "AnthropometryBodySize" "AnthropometryImpedance" "ArterialStiffness" "BloodBiochemistry" "BloodCount" "BloodPressure" "BoneDensitometryOfHeel" "BrainGreyMatterVolumes" "BrainSubcorticalVolumes" "BraindMRIWeightedMeans" "Breathing" "CancerScreening" "CarotidUltrasound" "ChestPain" "Claudication" "CognitiveFluidIntelligence" "CognitiveMatrixPatternCompletion" "CognitiveNumericMemory" "CognitivePairedAssociativeLearning" "CognitivePairsMatching" "CognitiveProspectiveMemory" "CognitiveReactionTime" "CognitiveSymbolDigitSubstitution" "CognitiveTowerRearranging" "CognitiveTrailMaking" "Diet" "ECGAtRest" "EarlyLifeFactors" "Education" "ElectronicDevices" "Employment" "EyeAcuity" "EyeAutorefraction" "EyeIntraocularPressure" "Eyesight" "FamilyHistory" "GeneralHealth" "GeneralPain" "HandGripStrength" "Hearing" "HearingTest" "HeartPWA" "HeartSize" "Household" "Medication" "MentalHealth" "Mouth" "OtherSociodemographics" "PhysicalActivity" "SexualFactors" "Sleep" "Smoking" "SocialSupport